In [10]:
import json
import csv
import random
import time
import torch
import torch.nn as nn
import numpy as np

In [101]:

fasttext_sv = np.load('data/snips_processed/FastText-sv.npy')
fasttext_en = np.load('data/snips_processed/FastText-en.npy')
fasttext_uw_sv = np.load('data/snips_processed/FastText-uw-sv.npy')
fasttext_uw_en = np.load('data/snips_processed/FastText-uw-en.npy')

elmo_sv = np.load('data/snips_processed/ELMO-sv.npy')
elmo_en = np.load('data/snips_processed/ELMO-en.npy')
use_en = np.load('data/snips_processed/USE-en.npy')

labs = np.load('data/snips_processed/labs.npy')

fasttext_sv = np.squeeze(fasttext_sv, axis=1)
fasttext_en = np.squeeze(fasttext_en, axis=1)
fasttext_uw_sv = np.squeeze(fasttext_uw_sv, axis=1)
fasttext_uw_en = np.squeeze(fasttext_uw_en, axis=1)
elmo_sv = np.squeeze(elmo_sv, axis=1)
elmo_en = np.squeeze(elmo_en, axis=1)
use_en = np.squeeze(use_en, axis=1)


#appendd bias
fasttext_sv = np.hstack([fasttext_sv, np.array([np.ones(len(fasttext_sv))]).T])
fasttext_uw_sv = np.hstack([fasttext_uw_sv, np.array([np.ones(len(fasttext_uw_sv))]).T])
elmo_sv = np.hstack([elmo_sv, np.array([np.ones(len(elmo_sv))]).T])


In [90]:
def closest_index(request, dots, forbiden_index=-1):
    dists = np.linalg.norm(dots-request, axis=1)
    res =  np.argmin(dists)
    
    if res == forbiden_index:
        dists[res] = np.inf
        return np.argmin(dists)
    return res

def test_transformation(x, test_in, test_out, test_labels, verbose = False, notransform=False):
    right = 0
    n =  len(test_in)
    for i in range(n):
        vec_in = test_in[i]
        true_lab = test_labels[i]
        
        vec_out =  vec_in if notransform else vec_in@x
        predicted_index = closest_index(vec_out, test_out, forbiden_index=i)
        predicted_lab = test_labels[predicted_index]
        
        if predicted_lab == true_lab:
            right += 1
            
        if verbose and i%100 == 0:
            print('{}/{} tested, right: {}, acc {}'.format(i, n, right, right/n))

    return right/n
        
        
    
def test_transform_pair(vecs_in, vecs_out, labels, start_test_i=0,end_test_i=5000, notransform=False):
    
    train_in = np.vstack((vecs_in[:start_test_i], vecs_in[end_test_i :]))
    train_out = np.vstack((vecs_out[:start_test_i], vecs_out[end_test_i :]))
    
    test_in = vecs_in[start_test_i: end_test_i]
    test_out = vecs_out[start_test_i: end_test_i]
    test_labels = labels[start_test_i: end_test_i]
    
    x, res, rank, sing = np.linalg.lstsq(train_in, train_out)
    
    return test_transformation(x, test_in, test_out, test_labels, notransform=notransform)
        

In [91]:
def cv(vecs_in, vecs_out, labs, folds = 5, notransform=False):
    
    delims = np.arange(0, len(vecs_in), len(vecs_in)//folds)
    results = []
    t = time.time()
    for i in range(folds):
        acc = test_transform_pair(vecs_in, vecs_out,labs,  start_test_i=delims[i],end_test_i=delims[i+1], notransform=notransform)
        
        results.append(acc)
        print('#{:3d}, {:5d} sec. acc = {:.3f}'.format(i+1, int(time.time() - t), results[-1]))

    return(sum(results)/len(results))   
    

In [71]:
cv(elmo_sv, elmo_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  0,    52 sec. acc = 0.901
#  1,   104 sec. acc = 0.897
#  2,   158 sec. acc = 0.897
#  3,   208 sec. acc = 0.872
#  4,   260 sec. acc = 0.892


0.891944847605225

In [81]:
cv(fasttext_sv, elmo_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  1,    86 sec. acc = 0.882
#  2,   158 sec. acc = 0.873
#  3,   217 sec. acc = 0.874
#  4,   264 sec. acc = 0.835
#  5,   310 sec. acc = 0.849


0.8626269956458635

In [73]:
cv(elmo_sv, use_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  0,    24 sec. acc = 0.910
#  1,    48 sec. acc = 0.923
#  2,    72 sec. acc = 0.919
#  3,    93 sec. acc = 0.904
#  4,   115 sec. acc = 0.923


0.9158925979680695

In [82]:
cv(fasttext_sv, use_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  1,    22 sec. acc = 0.890
#  2,    42 sec. acc = 0.907
#  3,    60 sec. acc = 0.906
#  4,    79 sec. acc = 0.880
#  5,    98 sec. acc = 0.899


0.896589259796807

In [85]:
cv(fasttext_uw_sv, elmo_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  1,    46 sec. acc = 0.903
#  2,    96 sec. acc = 0.898
#  3,   145 sec. acc = 0.890
#  4,   188 sec. acc = 0.861
#  5,   261 sec. acc = 0.892


0.8888243831640057

In [84]:
cv(fasttext_uw_sv, use_en, labs)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  1,    22 sec. acc = 0.910
#  2,    40 sec. acc = 0.926
#  3,    60 sec. acc = 0.918
#  4,    81 sec. acc = 0.890
#  5,   107 sec. acc = 0.918


0.9123367198838895

In [103]:
#notransform (single emb test)

#cv(fasttext_uw_en, fasttext_uw_en,labs, notransform=True)
print(cv(fasttext_en, fasttext_en,labs, notransform=True))
print(cv(fasttext_sv, fasttext_sv,labs, notransform=True))
print(cv(fasttext_uw_sv, fasttext_uw_sv,labs, notransform=True))
print(cv(elmo_sv, elmo_sv,labs, notransform=True))
print(cv(elmo_en, elmo_en,labs, notransform=True))

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:40: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


#  1,     6 sec. acc = 0.861
#  2,    15 sec. acc = 0.870
#  3,    24 sec. acc = 0.849
#  4,    31 sec. acc = 0.848
#  5,    39 sec. acc = 0.849
0.8555152394775035
#  1,    14 sec. acc = 0.814
#  2,    32 sec. acc = 0.817
#  3,    47 sec. acc = 0.817
#  4,    62 sec. acc = 0.806
#  5,    78 sec. acc = 0.798
0.8103047895500726
#  1,    16 sec. acc = 0.903
#  2,    32 sec. acc = 0.901
#  3,    47 sec. acc = 0.902
#  4,    61 sec. acc = 0.901
#  5,    75 sec. acc = 0.889
0.8992017416545718
#  1,    54 sec. acc = 0.895
#  2,   108 sec. acc = 0.904
#  3,   154 sec. acc = 0.904
#  4,   201 sec. acc = 0.892
#  5,   250 sec. acc = 0.901
0.8991291727140783
#  1,    25 sec. acc = 0.922
#  2,    51 sec. acc = 0.915
#  3,    76 sec. acc = 0.911
#  4,   102 sec. acc = 0.913
#  5,   130 sec. acc = 0.909
0.9140058055152395


In [137]:
print('Transform matrix for w2v {}, \nTransform m for elmo {}'.format(x.shape, x_e.shape))
print('train error of regression for w2w {:.3}, elmo {:.3}, \ntest error w2v {:.3} elmo {:.3}'.format(
    np.linalg.norm(en_train -(sv_train@x)),\
    np.linalg.norm(en_train -(sv_train_e@x_e)), \
    np.linalg.norm(en_val -(sv_val@x)),\
    np.linalg.norm(en_val -(sv_val_e@x_e)))) 


Transform matrix for w2v (301, 512), 
Transform m for elmo (1025, 512)
train error of regression for w2w 54.7, elmo 40.7, 
test error w2v 3.59e+10 elmo 57.0


In [17]:
def eval(model, vectors, labels):
    with torch.no_grad():
        vectors = list(map(lambda x: np.array([x]) , vectors))
        #labels = list(map(lambda x: np.array([x]) , labels))
        vectors = torch.tensor(list(vectors)).float()
        labels = torch.tensor(labels)
            
        model_out = model.forward(vectors)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1
        return right/len(model_out)
    
print('USE vectors on USE model testset: {:.3}'.format(eval(net, en_test, labs_test)))
print('Transformed w2v vecs from val and test {:.3}, {:.3}'.format(eval(net, sv_val@x, labs_val),
                                                             eval(net, sv_test@x, labs_test)))
print('Transformed elmo vecs from val and test {:.3}, {:.3}'.format(eval(net, sv_val_e@x_e, labs_val),
                                                             eval(net, sv_test_e@x_e, labs_test)))


USE vectors on USE model testset: 0.153


NameError: name 'x' is not defined

In [9]:
vecs_en

NameError: name 'vecs_en' is not defined

In [76]:
len(use_en[1])


512

In [14]:
#np.hstack(
fasttext_sv.shape, np.array([np.ones(len(fasttext_sv))]).T.shape
#)

((13784, 1, 300), (13784, 1))